In [1]:
!pip install pandas itables torch transformers scikit-learn tqdm matplotlib torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import pandas as pd
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import tqdm as notebook_tqdm
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn.functional as F
import seaborn as sns
import torchmetrics
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [1]:
for i in range(8):
    print(i)

0
1
2
3
4
5
6
7


In [3]:
df_train = pd.read_csv('labeled_dataset.csv')
df_train.head(5)

translated_sentence sentiment
0     Excellent as always, especially Italian burger  Positive
1  Dude I don't have any cutlery! Am I supposed t...  Negative
2  It was very tasty I wish you had more variety,...  Positive
3  It was good the only thing was the size of the...  Positive
4  On time and good only the courier didn't bring...  Negative

In [4]:
df_train = df_train[df_train['sentiment'].isin(['Negative', 'Positive'])]

In [5]:
texts = df_train["translated_sentence"].tolist()
labels = [1 if sentiment == "Positive" else 0 for sentiment in df_train["sentiment"].tolist()]

In [6]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            return_tensors='pt',
            max_length=self.max_length,
            padding='max_length',
            truncation=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label)
        }

In [22]:
class RobertaClassifier(nn.Module):
    def __init__(self, roberta_model_name, num_classes):
        super(RobertaClassifier, self).__init__()
        self.roberta = RobertaModel.from_pretrained(roberta_model_name)

        self.dropout1 = nn.Dropout(0.2)
        self.fc1 = nn.Linear(self.roberta.config.hidden_size, 512)
        self.gelu1 = nn.GELU()
        self.layernorm1 = nn.LayerNorm(512)

        self.dropout2 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 256)
        self.gelu2 = nn.GELU()
        self.layernorm2 = nn.LayerNorm(256)

        self.dropout3 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]

        x = self.fc1(pooled_output)
        x = self.gelu1(x)
        x = self.layernorm1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.gelu2(x)
        x = self.layernorm2(x)
        x = self.dropout2(x)

        logits = self.fc3(x)
        return logits

In [8]:
def train(model, data_loader, optimizer, scheduler, device, weight):
    model.train()
    total_loss = 0
    train_losses = []
    train_accuracies = []

    weight_np = np.array(weight)
    classes = np.array([0, 1])
    class_weights = compute_class_weight('balanced', classes=classes, y=weight_np)
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

    criterion = nn.CrossEntropyLoss(weight=class_weights)

    loop = tqdm(data_loader, desc="Training", leave=True)

    for batch in loop:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = criterion(outputs, labels)
        total_loss += loss.item()
        train_losses.append(loss.item())

        _, preds = torch.max(outputs, dim=1)
        batch_acc = (preds == labels).float().mean().item()
        train_accuracies.append(batch_acc)

        loss.backward()
        optimizer.step()

        if scheduler:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(loss)
            else:
                scheduler.step()

        loop.set_postfix(loss=loss.item(), acc=batch_acc)

    avg_loss = total_loss / len(data_loader)
    avg_accuracy = sum(train_accuracies) / len(train_accuracies)
    print(f"\nEpoch Loss: {avg_loss:.4f} | Epoch Accuracy: {avg_accuracy:.4f}")

    return train_losses, train_accuracies

In [9]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions, actual_labels = [], []
    eval_losses = []
    eval_accuracies = []

    loop = tqdm(data_loader, desc="Evaluating", leave=True)
    with torch.no_grad():
        for batch in loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            loss = F.cross_entropy(outputs, labels)
            eval_losses.append(loss.item())

            _, preds = torch.max(outputs, dim=1)
            batch_acc = (preds == labels).float().mean().item()
            eval_accuracies.append(batch_acc)

            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())

            loop.set_postfix(loss=loss.item(), acc=batch_acc)

    avg_loss = sum(eval_losses) / len(eval_losses)
    avg_accuracy = sum(eval_accuracies) / len(eval_accuracies)
    report = classification_report(actual_labels, predictions, digits=4)

    print(f"\nValidation Loss: {avg_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}")
    return avg_loss, avg_accuracy, eval_losses, eval_accuracies, report

In [42]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()

    encoding = tokenizer(text, return_tensors='pt', max_length=max_length,
                         padding='max_length', truncation=True)

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = F.softmax(outputs, dim=1)
        preds = torch.argmax(probabilities, dim=1)

    sentiment = "positive" if preds.item() == 1 else "negative"
    confidence = probabilities[0, preds.item()].item()
    print(f"Sentiment: {sentiment} (Confidence: {confidence:.2f})")

    return sentiment, confidence

In [12]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [18]:
roberta_model_name = 'roberta-base'
num_classes = 2
max_length = 256
batch_size = 32
num_epochs = 4
learning_rate = 2e-5

In [14]:
tokenizer = RobertaTokenizer.from_pretrained(roberta_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaClassifier(roberta_model_name, num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
total_steps = len(train_dataloader) * num_epochs
warmup_steps = int(0.1 * total_steps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
gradient_accumulation_steps = 2

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss, train_acc = train(model, train_dataloader, optimizer, scheduler, device, weight=labels)

    val_loss, val_acc, eval_losses, eval_accuracies, report = evaluate(model, val_dataloader, device)

    print(val_acc)
    print(report)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

Epoch 1/4


Training: 100%|██████████| 1377/1377 [30:16<00:00,  1.32s/it, acc=1, loss=0.0188]



Epoch Loss: 0.2191 | Epoch Accuracy: 0.8998


Evaluating: 100%|██████████| 345/345 [02:23<00:00,  2.41it/s, acc=1, loss=0.0274]



Validation Loss: 0.1427 | Validation Accuracy: 0.9373
0.9373188405797102
              precision    recall  f1-score   support

           0     0.9752    0.9198    0.9467      6681
           1     0.8862    0.9640    0.9234      4329

    accuracy                         0.9371     11010
   macro avg     0.9307    0.9419    0.9351     11010
weighted avg     0.9402    0.9371    0.9375     11010

Epoch 2/4


...


Validation Loss: 0.1387 | Validation Accuracy: 0.9451
0.9451086956521739
              precision    recall  f1-score   support

           0     0.9704    0.9379    0.9539      6681
           1     0.9089    0.9559    0.9318      4329

    accuracy                         0.9450     11010
   macro avg     0.9396    0.9469    0.9428     11010
weighted avg     0.9462    0.9450    0.9452     11010

Epoch 4/4


Training: 100%|██████████| 1377/1377 [30:20<00:00,  1.32s/it, acc=1, loss=0.00222]



Epoch Loss: 0.0759 | Epoch Accuracy: 0.9708


Evaluating: 100%|██████████| 345/345 [02:23<00:00,  2.41it/s, acc=1, loss=0.00195]


Validation Loss: 0.1497 | Validation Accuracy: 0.9477
0.9477355072463768
              precision    recall  f1-score   support

           0     0.9651    0.9479    0.9564      6681
           1     0.9218    0.9471    0.9343      4329

    accuracy                         0.9476     11010
   macro avg     0.9434    0.9475    0.9453     11010
weighted avg     0.9481    0.9476    0.9477     11010



In [26]:
torch.save(model.state_dict(), "roberta_classifier.pth")

In [31]:
test_text = "i dont like food so bad ."
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print(test_text)
print(f"Predicted sentiment: {sentiment}")

Sentiment: negative (Confidence: 0.99)
i dont like food so bad .
Predicted sentiment: ('negative', 0.9948636889457703)


In [36]:
def predict_positive_probability(text, model, tokenizer, device, max_length=128):
    model.eval()

    encoding = tokenizer(text, return_tensors='pt', max_length=max_length,
                         padding='max_length', truncation=True)

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)


        logits = outputs
        probabilities = F.softmax(logits, dim=1)

        positive_prob = probabilities[0, 1].item()

    return positive_prob

In [41]:
test_text = "food so good ."
sentiment = predict_positive_probability(test_text, model, tokenizer, device)
print(test_text)
print(f"Predicted sentiment: {sentiment}")

food so good .
Predicted sentiment: 0.9998973608016968


In [58]:
df_test = pd.read_csv('labeled_test_dataset.csv')
df_test.head(5)

translated_sentence sentiment
0                The sauce of the pasta was not good  Negative
1  Shahrojan, I started getting really hopeless l...  Negative
2  The drink was not delivered with my food. The ...  Negative
3  This restaurant is really great. I've been ord...  Positive
4  The quality of Farnaz sweets was very poor com...  Negative

In [46]:
df_test['prediction'] = df_test['translated_sentence'].apply(lambda x: predict_positive_probability(x, model, tokenizer, device))

In [47]:
df_test

translated_sentence sentiment  prediction
0                    The sauce of the pasta was not good  Negative    0.003442
1      Shahrojan, I started getting really hopeless l...  Negative    0.000980
2      The drink was not delivered with my food. The ...  Negative    0.000634
3      This restaurant is really great. I've been ord...  Positive    0.999908
4      The quality of Farnaz sweets was very poor com...  Negative    0.002128
...                                                  ...       ...         ...
13995                           The potato was excellent  Positive    0.999918
13996   The volume, quality and packaging have decreased  Negative    0.001941
13997  As always the quality of your food was excelle...  Negative    0.014334
13998                     Delicious, quality and healthy  Positive    0.999925
13999  It is suggested that due care be taken in hand...  Positive    0.999509

[14000 rows x 3 columns]

In [49]:
df_test['prediction'].to_csv('output.csv', index=False)

In [50]:
df_test['sentiment_model'] = df_test['translated_sentence'].apply(lambda x: predict_sentiment(x, model, tokenizer, device))

Streaming output truncated to the last 5000 lines.
Sentiment: negative (Confidence: 0.99)
Sentiment: negative (Confidence: 1.00)
Sentiment: positive (Confidence: 1.00)
Sentiment: negative (Confidence: 1.00)
Sentiment: positive (Confidence: 1.00)
Sentiment: negative (Confidence: 1.00)
Sentiment: negative (Confidence: 1.00)
Sentiment: negative (Confidence: 1.00)
Sentiment: positive (Confidence: 1.00)
Sentiment: positive (Confidence: 1.00)
Sentiment: positive (Confidence: 1.00)
Sentiment: positive (Confidence: 0.75)
Sentiment: negative (Confidence: 1.00)
Sentiment: positive (Confidence: 1.00)
Sentiment: positive (Confidence: 1.00)
Sentiment: negative (Confidence: 1.00)
Sentiment: positive (Confidence: 0.99)
Sentiment: positive (Confidence: 1.00)
Sentiment: negative (Confidence: 1.00)
Sentiment: negative (Confidence: 1.00)
Sentiment: negative (Confidence: 1.00)
Sentiment: positive (Confidence: 1.00)
Sentiment: positive (Confidence: 0.58)
Sentiment: negative (Confidence: 0.99)
Sentiment: po

In [51]:
df_test

translated_sentence sentiment  \
0                    The sauce of the pasta was not good  Negative   
1      Shahrojan, I started getting really hopeless l...  Negative   
2      The drink was not delivered with my food. The ...  Negative   
3      This restaurant is really great. I've been ord...  Positive   
4      The quality of Farnaz sweets was very poor com...  Negative   
...                                                  ...       ...   
13995                           The potato was excellent  Positive   
13996   The volume, quality and packaging have decreased  Negative   
13997  As always the quality of your food was excelle...  Negative   
13998                     Delicious, quality and healthy  Positive   
13999  It is suggested that due care be taken in hand...  Positive   

       prediction                 sentiment_model  
0        0.003442  (negative, 0.9965575933456421)  
1        0.000980  (negative, 0.9990196228027344)  
2        0.000634  (negative, 0.9993658661842346)  
3        0.999908  (positive, 0.9999076128005981)  
4        0.002128  (negative, 0.9978715181350708)  
...           ...                             ...  
13995    0.999918  (positive, 0.9999181032180786)  
13996    0.001941  (negative, 0.9980594515800476)  
13997    0.014334   (negative, 0.985665500164032)  
13998    0.999925  (positive, 0.9999253749847412)  
13999    0.999509  (positive, 0.9995087385177612)  

[14000 rows x 4 columns]